# Getting Data

In [1]:
from decouple import Config,RepositoryEnv
env_path = '/home/sfniila/.ssh/.env'
env_config = Config(RepositoryEnv(env_path))
github_token = env_config.get('GITHUB_TOKEN')

## PyGitHub

In [2]:
from github import Github

def pygithub_get_repo_paths(
    token: str,
    owner: str, 
    name: str
) -> any:
    g = Github(token)
    repo = g.get_repo(f"{owner}/{name}")
    contents = repo.get_contents("")
    paths = []
    while len(contents) > 0:
      file_content = contents.pop(0)
      if file_content.type == 'dir':
        contents.extend(repo.get_contents(file_content.path))
      else:
        paths.append(file_content.path)
    g.close()
    return paths

def pygithub_get_path_content(
    token: str,
    owner: str, 
    name: str, 
    path: str
) -> any:
    g = Github(token)
    repo = g.get_repo(f"{owner}/{name}")
    file_content = repo.get_contents(path)
    content = file_content.decoded_content.decode('utf-8')
    # fails on some yaml files
    # deployment/kubeflow/manifests/contrib/ray/kuberay-operator/base/resources.yaml
    # unsupported encoding: none
    g.close()
    return content

## Scraping Functions

In [3]:
import os

def get_github_repo_documents(
    github_token: str,
    repo_owner: str,
    repo_name: str,
    relevant_files: any,
    store: bool
) -> any:
    storage_path = os.getcwd() + '/' + repo_owner + '-' + repo_name + '.txt' 

    repo_paths = []
    if not os.path.exists(storage_path):
        print('Getting github paths')
        repo_paths = pygithub_get_repo_paths(
            token = github_token,
            owner = repo_owner, 
            name = repo_name
        )
        
        if store:
            print('Storing paths')
            path_amount = len(repo_paths)
            i = 0
            with open(storage_path, 'w') as file:
                for line in repo_paths:
                    row = line 
                    if i < path_amount:
                        row += '\n'
                    file.write(row)
                    i += 1
            print('Paths stored')
    else:
        print('Getting stored paths')
        with open(storage_path, 'r') as file:
            repo_paths = file.readlines()
    print('Paths fetched')
                
    print('Filtering paths')
    relevant_paths = []
    for path in repo_paths:
        path_split = path.split('/')
        file_end = path_split[-1].split('.')[-1].rstrip()
        if file_end in relevant_files:
            relevant_paths.append(path.rstrip())
    print('Paths filtered')

    formatted_paths = {
        'paths': relevant_paths
    }

    # consider how to store this to allas
    return formatted_paths

Files of intrest
- md = important to usesr
- txt = isn't important to users
- sh = isn't important to users
- yaml = important to developers
- py = important to users and developers
- ipynb = important to users and developers

In [4]:
repository_paths = get_github_repo_documents(
    github_token = github_token,
    repo_owner = 'K123AsJ0k1',
    repo_name = 'cloud-hpc-oss-mlops-platform',
    relevant_files = [
        'md',
        'yaml',
        'py',
        'ipynb'
    ],
    store = True
)

Getting stored paths
Paths fetched
Filtering paths
Paths filtered


# Storing Data

## Mongo Functions

In [5]:
from pymongo import MongoClient as mc

def mongo_is_client(
    storage_client: any
) -> any:
    return isinstance(storage_client, mc.Connection)

def mongo_setup_client(
    username: str,
    password: str,
    address: str,
    port: str
) -> any:
    connection_prefix = 'mongodb://(username):(password)@(address):(port)/'
    connection_address = connection_prefix.replace('(username)', username)
    connection_address = connection_address.replace('(password)', password)
    connection_address = connection_address.replace('(address)', address)
    connection_address = connection_address.replace('(port)', port)
    mongo_client = mc(
        host = connection_address
    )
    return mongo_client

def mongo_get_database(
    mongo_client: any,
    database_name: str
) -> any:
    try:
        database = mongo_client[database_name]
        return database
    except Exception as e:
        return None

def mongo_check_database(
    mongo_client: any, 
    database_name: str
) -> bool:
    try:
        database_exists = database_name in mongo_client.list_database_names()
        return database_exists
    except Exception as e:
        return False

def mongo_list_databases(
    mongo_client: any
) -> any:
    try:
        databases = mongo_client.list_database_names()
        return databases
    except Exception as e:
        return []

def mongo_remove_database(
    mongo_client: any, 
    database_name: str
) -> bool:
    try:
        mongo_client.drop_database(database_name)
        return True
    except Exception as e:
        return False

def mongo_get_collection(
    mongo_client: any, 
    database_name: str, 
    collection_name: str
) -> bool:
    try:
        database = mongo_get_database(
            mongo_client = mongo_client,
            database_name = database_name
        )
        collection = database[collection_name]
        return collection
    except Exception as e:
        return None
    
def mongo_check_collection(
    mongo_client: any, 
    database_name: any, 
    collection_name: any
) -> bool:
    try:
        database = mongo_client[database_name]
        collection_exists = collection_name in database.list_collection_names()
        return collection_exists
    except Exception as e:
        return False

def mongo_update_collection(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any, 
    update_query: any
) -> any:
    try:
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.update_many(filter_query, update_query)
        return result
    except Exception as e:
        return None

def mongo_list_collections(
    mongo_client: any, 
    database_name: str
) -> bool:
    try:
        database = mongo_get_database(
            mongo_client = mongo_client,
            database_name = database_name
        )
        collections = database.list_collection_names()
        return collections
    except Exception as e:
        return []

def mongo_remove_collection(
    mongo_client: any, 
    database_name: str, 
    collection_name: str
) -> bool:
    try: 
        database = mongo_get_database(
            mongo_client = mongo_client,
            database_name = database_name
        )
        database.drop_collection(collection_name)
        return True
    except Exception as e:
        return False

def mongo_create_document(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    document: any
) -> any:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.insert_one(document)
        return result
    except Exception as e:
        return None

def mongo_get_document(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any
):
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        document = collection.find_one(filter_query)
        return document
    except Exception as e:
        print(e)
        return None 

def mongo_list_documents(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any,
    sorting_query: any
) -> any:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        documents = list(collection.find(filter_query).sort(sorting_query))
        return documents
    except Exception as e:
        return []

def mongo_update_document(
    mongo_client: any, 
    database_name: any, 
    collection_name: any, 
    filter_query: any, 
    update_query: any
) -> any:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.update_one(filter_query, update_query)
        return result
    except Exception as e:
        return None

def mongo_remove_document(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any
) -> bool:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.delete_one(filter_query)
        return result
    except Exception as e:
        return None

## Markdown documents

In [6]:
import markdown
from bs4 import BeautifulSoup

def create_markdown_documents(
    markdown_text: any
) -> any:
    html = markdown.markdown(markdown_text)
    soup = BeautifulSoup(html, features='html.parser')
    code_block_pattern = re.compile(r"```")
    
    documents = []
    document = ''
    index = 1
    for element in soup.descendants:
        if element.name in ['h2', 'h3', 'h4', 'h5', 'h6']:
            text = element.get_text(strip = True)
            if not document == '':
                document = document.replace('\n', '')
                if not len(document.split()) == 1:
                    documents.append({
                        'index': index,
                        'sub-index': 0,
                        'type': 'markdown',
                        'data': document
                    })
                    index += 1
                document = ''
            document += text
        elif element.name == 'p':
            text = element.get_text(strip = True)
            text = re.sub(code_block_pattern, '', text)
            text = text.rstrip('\n')
            text = text.replace('\nsh', '')
            text = text.replace('\nbash', '')
            document += ' ' + text
        elif element.name in ['ul', 'ol']:
            text = ''
            for li in element.find_all('li'):
                item = li.get_text(strip=True)
                if not '-' in item:
                    text += '-' + item
                    continue
                text += item
            document += ' ' + text
            
    documents.append({
        'index': index,
        'sub-index': 0,
        'type': 'markdown',
        'data': document
    })
    
    formatted_documents = {
        'text': documents
    }
    
    return formatted_documents

In [83]:
example_data = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = repository_paths[7]
)

In [163]:
markdown_documents = create_markdown_documents(
    markdown_text = example_data
) 

## YAML documents

In [7]:
import yaml 

def extract_yaml_values(
    section: any,
    path: str,
    values: any
) -> any:
    for key, value in section.items():
        if path == '':
            current_path = key
        else:
            current_path = path + '/' + key
        if isinstance(value, dict):
            extract_yaml_values(
                section = value,
                path = current_path,
                values = values
            )
        if isinstance(value, list):
            number = 1
            
            for case in value:
                base_path = current_path
                if isinstance(case, dict):
                   extract_yaml_values(
                       section = case,
                       path = current_path,
                       values = values
                   ) 
                   continue
                base_path += '/' + str(number)
                number += 1
                values.append(base_path + '=' + str(case))
        else:
            if isinstance(value, dict):
                continue
            if isinstance(value, list):
                continue
            values.append(current_path + '=' + str(value))
            
    return values

def create_yaml_documents(
    yaml_text: any
) -> any:
    # has problems with some yaml files
    # unsupported encoding: none
    # deployment/kubeflow/manifests/apps/kfp-tekton/upstream/v1/third-party/tekton/upstream/manifests/base/tektoncd-install/tekton-controller.yaml
    # 'list' object has no attribute 'items'
    # deployment/kubeflow/manifests/apps/kfp-tekton/upstream/v1/third-party/tekton/upstream/manifests/base/tektoncd-install/tekton-release.yaml
    # 'NoneType' object has no attribute 'items'
    # deployment/kubeflow/manifests/apps/kfp-tekton/upstream/v1/base/metadata/overlays/db/kustomization.yaml
    #   could not determine a constructor for the tag 'tag:yaml.org,2002:value'
    # in "<unicode string>", line 41, column 18:
    #      delimiter: =
    yaml_data = list(yaml.safe_load_all(yaml_text))

    documents = []
    index = 1
    for data in yaml_data:
        yaml_values = extract_yaml_values(
            section = data,
            path = '',
            values = []
        )

        previous_root = ''
        document = ''
        sub_index = 1
        for value in yaml_values:
            equal_split = value.split('=')
            path_split = equal_split[0].split('/')
            root = path_split[0]
            if not root == previous_root:
                if 0 < len(document):
                    documents.append({
                        'index': index,
                        'sub-index': sub_index,
                        'type': 'yaml',
                        'data': document
                    })
                    sub_index += 1
                    
                previous_root = root
                document = value
            else:
                document += value
                
        documents.append({
            'index': index,
            'sub-index': sub_index,
            'type': 'yaml',
            'data': document
        })
        index += 1

    formatted_documents = {
        'text': documents
    }
            
    return formatted_documents

In [61]:
example_yaml = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = 'deployment/forwarder/celery/celery-deployment.yaml'
)

In [62]:
yaml_documents = create_yaml_documents(
    yaml_text = example_yaml
) 

## TreeSitter Functions

In [8]:
import tree_sitter_python as tspython
from tree_sitter import Language, Parser
import re

def tree_extract_imports(
    node: any, 
    code_text: str
) -> any:
    imports = []
    if node.type == 'import_statement' or node.type == 'import_from_statement':
        start_byte = node.start_byte
        end_byte = node.end_byte
        imports.append(code_text[start_byte:end_byte].decode('utf8'))
    for child in node.children:
        imports.extend(tree_extract_imports(child, code_text))
    return imports

def tree_extract_dependencies(
    node: any, 
    code_text: str
) -> any:
    dependencies = []
    for child in node.children:
        if child.type == 'call':
            dependency_name = child.child_by_field_name('function').text.decode('utf8')
            dependencies.append(dependency_name)
        dependencies.extend(tree_extract_dependencies(child, code_text))
    return dependencies

def tree_extract_code_and_dependencies(
    node: any,
    code_text: str
) -> any:
    codes = []
    if not node.type == 'function_definition':
        start_byte = node.start_byte
        end_byte = node.end_byte
        name = node.child_by_field_name('name')
        if name is None:
            code = code_text[start_byte:end_byte].decode('utf8')
            if not 'def' in code:
                dependencies = tree_extract_dependencies(node, code_text)
                codes.append({
                    'name': 'global',
                    'code': code,
                    'dependencies': dependencies
                })
    return codes

def tree_extract_functions_and_dependencies(
    node: any, 
    code_text: str
) -> any:
    functions = []
    if node.type == 'function_definition':
        start_byte = node.start_byte
        end_byte = node.end_byte
        name = node.child_by_field_name('name').text.decode('utf8')
        code = code_text[start_byte:end_byte].decode('utf8')
        dependencies = tree_extract_dependencies(node, code_text)
        functions.append({
            'name': name,
            'code': code,
            'dependencies': dependencies
        })
    for child in node.children:
        functions.extend(tree_extract_functions_and_dependencies(child, code_text))
    return functions

def tree_get_used_imports(
    general_imports: any,
    function_dependencies: any
) -> any:
    parsed_imports = {}
    for code_import in general_imports:
        import_factors = code_import.split('import')[-1].replace(' ', '')
        import_factors = import_factors.split(',')
    
        for factor in import_factors:
            if not factor in parsed_imports:
                parsed_imports[factor] = code_import.split('import')[0] + 'import ' + factor
            
    relevant_imports = {}
    for dependency in function_dependencies:
        initial_term = dependency.split('.')[0]
    
        if not initial_term in relevant_imports:
            if initial_term in parsed_imports:
                relevant_imports[initial_term] = parsed_imports[initial_term]
    
    used_imports = []
    for name, code in relevant_imports.items():
        used_imports.append(code)

    return used_imports

def tree_get_used_functions(
    general_functions: any,
    function_dependencies: any
): 
    used_functions = []
    for related_function_name in function_dependencies:
        for function in general_functions:
            if function['name'] == related_function_name:
                used_functions.append('from ice import ' + function['name'])
    return used_functions

def tree_create_code_document(
    code_imports: any,
    code_functions: any,
    function_item: any
) -> any:
    used_imports = tree_get_used_imports(
        general_imports = code_imports,
        function_dependencies = function_item['dependencies']
    )

    used_functions = tree_get_used_functions(
        general_functions = code_functions,
        function_dependencies = function_item['dependencies']
    )
    
    document = {
        'imports': used_imports,
        'functions': used_functions,
        'name': function_item['name'],
        'dependencies': function_item['dependencies'],
        'code': function_item['code']
    }
    
    return document
     
def tree_format_code_document(
    code_document: any
) -> any:
    formatted_document = ''
    for doc_import in code_document['imports']:
        formatted_document += doc_import + '\n'

    for doc_functions in code_document['functions']:
        formatted_document += doc_functions + '\n'

    if 0 < len(code_document['dependencies']):
        formatted_document += 'code dependencies\n'

        for doc_dependency in code_document['dependencies']:
            formatted_document += doc_dependency + '\n'

    if code_document['name'] == 'global':
        formatted_document += code_document['name'] + ' code\n'
    else:
        formatted_document += 'function ' + code_document['name'] + ' code\n'
    
    for line in code_document['code'].splitlines():
        if not bool(line.strip()):
            continue
        doc_code = re.sub(r'#.*','', line)
        if not bool(doc_code.strip()):
            continue
        formatted_document += doc_code + '\n'    
    return formatted_document

def tree_create_python_code_and_function_documents(
    code_document: any
):
    PY_LANGUAGE = Language(tspython.language())
    parser = Parser(PY_LANGUAGE)
   
    tree = parser.parse(
        bytes(
            code_document,
            "utf8"
        )
    )

    root_node = tree.root_node
    code_imports = tree_extract_imports(
        root_node, 
        bytes(
            code_document, 
            'utf8'
        )
    )

    code_global = tree_extract_code_and_dependencies(
        root_node, 
        bytes(
            code_document, 
            'utf8'
        )
    )

    code_functions = tree_extract_functions_and_dependencies(
        root_node, 
        bytes(
            code_document, 
            'utf8'
        )
    )
    
    initial_documents = []
    for item in code_global:
        document = tree_create_code_document(
            code_imports = code_imports,
            code_functions = code_functions,
            function_item = item
        )  
        initial_documents.append(document)

    for item in code_functions:
        document = tree_create_code_document(
            code_imports = code_imports,
            code_functions = code_functions,
            function_item = item
        )  
        initial_documents.append(document)

    formatted_documents = []
    seen_functions = []
    for document in initial_documents:
        if not document['name'] == 'global':
            if document['name'] in seen_functions:
                continue
        
        formatted_document = tree_format_code_document(
            code_document = document
        )

        formatted_documents.append(formatted_document)
        seen_functions.append(document['name'])
    return formatted_documents

## Python documents

In [9]:
def create_python_documents(
    python_text: any
): 
    joined_code = ''.join(python_text)
    block_code_documents = tree_create_python_code_and_function_documents(
        code_document = joined_code
    )

    code_documents = []
    seen_function_names = []
    code_doc_index = 0
    for code_doc in block_code_documents:
        row_split = code_doc.split('\n')
        for row in row_split:
            if 'function' in row and 'code' in row:
                # This causes problems with some documents
                # list index out of range
                function_name = row.split(' ')[1]
                if not function_name in seen_function_names:
                    seen_function_names.append(function_name)
                else:
                    del block_code_documents[code_doc_index]
        code_doc_index += 1

    if 0 < len(block_code_documents):
        index = 1
        for code_doc in block_code_documents:
            code_documents.append({
                'index': index,
                'sub-index': 0,
                'type': 'python',
                'data': code_doc
            })
            index += 1
        
    formatted_documents = {
        'code': code_documents
    }
    return formatted_documents

In [65]:
example_python = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = 'applications/article/submitter/backend/functions/platforms/flower.py'
)

In [74]:
python_documents = create_python_documents(
    python_text = example_python
)

## Notebook documents

In [10]:
import nbformat
from bs4 import BeautifulSoup
import markdown

def parse_jupyter_notebook_markdown_into_text(
    markdown_text: any
) -> any:
    html = markdown.markdown(markdown_text)
    soup = BeautifulSoup(html, features='html.parser')
    text = soup.get_text()
    code_block_pattern = re.compile(r"```")
    text = re.sub(code_block_pattern, '', text)
    text = text.rstrip('\n')
    text = text.replace('\nsh', '\n')
    text = text.replace('\nbash', '\n')
    return text

def extract_jupyter_notebook_markdown_and_code(
    notebook_text: any
): 
    notebook_documents = {
        'markdown': [],
        'code': []
    }

    notebook = nbformat.reads(notebook_text, as_version=2)
    index = 1
    for cell in notebook.worksheets[0].cells:
        if cell.cell_type == 'markdown':
            notebook_documents['markdown'].append({
                'id': index,
                'data': cell.source
            })
            index += 1
        if cell.cell_type == 'code':
            notebook_documents['code'].append({
                'id': index,
                'data': cell.input
            })
            index += 1
    
    return notebook_documents

def create_notebook_documents(
    notebook_text: any
):
    notebook_documents = extract_jupyter_notebook_markdown_and_code(
        notebook_text = notebook_text
    )

    markdown_documents = []
    for block in notebook_documents['markdown']:
        joined_text = ''.join(block['data'])
        markdown_text = parse_jupyter_notebook_markdown_into_text(
            markdown_text = joined_text
        )
        markdown_documents.append({
            'index': block['id'],
            'sub-index': 0,
            'type': 'markdown',
            'data': markdown_text
        })
        
    code_documents = []
    seen_function_names = []
    for block in notebook_documents['code']:
        joined_code = ''.join(block['data'])
        block_code_documents = tree_create_python_code_and_function_documents(
            code_document = joined_code
        )

        code_doc_index = 0
        for code_doc in block_code_documents:
            row_split = code_doc.split('\n')
            for row in row_split:
                if 'function' in row and 'code' in row:
                    # This causes problems with some documents
                    # list index out of range
                    function_name = row.split(' ')[1]
                    if not function_name in seen_function_names:
                        seen_function_names.append(function_name)
                    else:
                        del block_code_documents[code_doc_index]
            code_doc_index += 1
        
        if 0 < len(block_code_documents):
            sub_indexes = False
            if 1 < len(block_code_documents):
                sub_indexes = True
            index = 1
            for code_doc in block_code_documents:
                if sub_indexes:
                    code_documents.append({
                        'index': block['id'],
                        'sub-index': index, 
                        'type': 'python',
                        'data': code_doc
                    })
                else:
                    code_documents.append({ 
                        'index': block['id'],
                        'sub-index': 0,
                        'type': 'python',
                        'data': code_doc
                    })
                index += 1
            
    formatted_documents = {
        'text': markdown_documents,
        'code': code_documents
    }
    
    return formatted_documents

In [76]:
example_notebook = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = 'tutorials/cloud_hpc/Cloud-HPC-FMNIST.ipynb'
)

In [97]:
notebook_documents = create_notebook_documents(
    notebook_text = example_notebook
)

## Storage Functions

In [11]:
mongo_client = mongo_setup_client(
    username = 'mongo123',
    password = 'mongo456',
    address = '127.0.0.1',
    port = '27017'
)

In [12]:
def store_repository_path_documents(
    mongo_client: any,
    github_token: any,
    repository_owner: str,
    repository_name: str,
    repository_path: str,
    database_name: str,
    collection_name: str
):
    collection_exists = mongo_check_collection(
        mongo_client = mongo_client, 
        database_name = database_name, 
        collection_name = collection_name
    )

    if collection_exists:
        return False

    target_content = ''
    try:
        target_content = pygithub_get_path_content(
            token = github_token,
            owner = repository_owner, 
            name = repository_name, 
            path = repository_path
        )
    except Exception as e:
        print(repository_path)
        print(e)

    if target_content == '':
        return False

    path_split = repository_path.split('/')
    target_type = path_split[-1].split('.')[-1]
    
    target_documents = {}
    if target_type == 'md':
        try:
            target_documents = create_markdown_documents(
                markdown_text = target_content
            )
        except Exception as e:
            print(repository_path)
            print(e)
    if target_type == 'yaml':
        try:
            target_documents = create_yaml_documents(
                yaml_text = target_content
            )
        except Exception as e:
            print(repository_path)
            print(e)
    if target_type == 'py':
        try:
            target_documents = create_python_documents(
                python_text = target_content
            )
        except Exception as e:
            print(repository_path)
            print(e)
    if target_type == 'ipynb':
        try:
            target_documents = create_notebook_documents(
                notebook_text = target_content
            )
        except Exception as e:
            print(repository_path)
            print(e)
    if 0 < len(target_documents):
        for doc_type, docs in target_documents.items():
            for document in docs:
                result = mongo_create_document(
                    mongo_client = mongo_client,
                    database_name = database_name,
                    collection_name = collection_name,
                    document = document
                )
        return True
    return False

def get_github_storage_prefix(
    repository_owner: str,
    repository_name: str
) -> str:
    return repository_owner + '|' + repository_name + '|'

def store_github_repository_documents(
    mongo_client: any,
    github_token: str,
    repository_owner: str,
    repository_name: str,
    repository_paths: any
) -> any:
    print('Storing paths')
    paths = repository_paths['paths']
    for path in paths:
        path_split = path.split('/')
        document_database_name = get_github_storage_prefix(repository_owner, repository_name) + path_split[-1].split('.')[-1]
        
        document_collection_name = ''
        for word in path_split[:-1]:
            document_collection_name += word[:2] + '|'
        document_collection_name += path_split[-1].split('.')[0]

        stored = store_repository_path_documents(
            mongo_client = mongo_client,
            github_token = github_token,
            repository_owner = repository_owner,
            repository_name = repository_name,
            repository_path = path,
            database_name = document_database_name,
            collection_name = document_collection_name
        )
    print('Paths stored')

In [147]:
store_github_repository_documents(
    mongo_client = mongo_client,
    github_token = github_token,
    repository_owner = 'K123AsJ0k1', 
    repository_name = 'cloud-hpc-oss-mlops-platform', 
    repository_paths = repository_paths
)

Storing paths
applications/development/LLMs/pipeline/preprocessing/RAG-Development.ipynb
list index out of range
applications/development/LLMs/pipeline/preprocessing/RAG-pipeline.ipynb
list index out of range
applications/development/LLMs/pipeline/preprocessing/RAG_Preprocessing.ipynb
list index out of range
applications/development/LLMs/pipeline/preprocessing/scripts/documents.py
list index out of range
applications/development/LLMs/pipeline/preprocessing/scripts/python_parsing.py
list index out of range
applications/development/LLMs/pipeline/preprocessing/scripts/platforms/tree.py
list index out of range
deployment/kubeflow/manifests/contrib/ray/kuberay-operator/base/resources.yaml
unsupported encoding: none
deployment/kubeflow/manifests/common/knative/knative-eventing/base/upstream/eventing-core.yaml
'NoneType' object has no attribute 'items'
deployment/kubeflow/manifests/common/knative/knative-eventing/base/upstream/in-memory-channel.yaml
'NoneType' object has no attribute 'items'


In [13]:
from pymongo import ASCENDING, DESCENDING
    
def get_stored_documents(
    mongo_client: any,
    database_prefix: str
) -> any:
    storage_structure = {}
    database_list = mongo_list_databases(
        mongo_client = mongo_client
    )
    for database_name in database_list:
        if database_prefix in database_name:
            collection_list = mongo_list_collections(
                mongo_client = mongo_client,
                database_name = database_name
            )
            storage_structure[database_name] = collection_list
    
    storage_documents = {}
    for database_name, collections in storage_structure.items():
        if not database_name in storage_documents:
            storage_documents[database_name] = {}
        for collection_name in collections:
            collection_documents = mongo_list_documents(
                mongo_client = mongo_client,
                database_name = database_name,
                collection_name = collection_name,
                filter_query = {},
                sorting_query = [
                    ('index', ASCENDING),
                    ('sub-index', ASCENDING)
                ]
            )
            storage_documents[database_name][collection_name] = collection_documents
            
    return storage_documents

In [14]:
stored_documents = get_stored_documents(
    mongo_client = mongo_client,
    database_prefix = get_github_storage_prefix('K123AsJ0k1', 'cloud-hpc-oss-mlops-platform')
)

## MinIO Functions

In [58]:
import io
import pickle

def create_bucket(
    logger: any,
    minio_client: any,
    bucket_name: str
) -> bool:
    MINIO_CLIENT = minio_client 
    try:
        MINIO_CLIENT.make_bucket(
            bucket_name = bucket_name
        )
        return True
    except Exception as e:
        logger.error('MinIO bucket creation error')
        logger.error(e)
        return False
    
def check_bucket(
    logger: any,
    minio_client: any,
    bucket_name:str
) -> bool:
    MINIO_CLIENT = minio_client
    try:
        status = MINIO_CLIENT.bucket_exists(bucket_name = bucket_name)
        return status
    except Exception as e:
        logger.error('MinIO bucket checking error')
        logger.error(e)
        return False 
       
def delete_bucket(
    logger: any,
    minio_client: any,
    bucket_name:str
) -> bool:
    MINIO_CLIENT = minio_client
    try:
        MINIO_CLIENT.remove_bucket(
            bucket_name = bucket_name
        )
        return True
    except Exception as e:
        logger.error('MinIO bucket deletion error')
        logger.error(e)
        return False
# Works
def create_object(
    logger: any,
    minio_client: any,
    bucket_name: str, 
    object_path: str, 
    data: any,
    metadata: dict
) -> bool: 
    # Be aware that MinIO objects have a size limit of 1GB, 
    # which might result to large header error
    MINIO_CLIENT = minio_client
    
    pickled_data = pickle.dumps(data)
    length = len(pickled_data)
    buffer = io.BytesIO()
    buffer.write(pickled_data)
    buffer.seek(0)
    try:
        MINIO_CLIENT.put_object(
            bucket_name = bucket_name,
            object_name = object_path + '.pkl',
            data = buffer,
            length = length,
            metadata = metadata
        )
        return True
    except Exception as e:
        logger.error('MinIO object creation error')
        logger.error(e)
        return False
# Works
def check_object(
    logger: any,
    minio_client: any,
    bucket_name: str, 
    object_path: str
) -> bool: 
    MINIO_CLIENT = minio_client
    try:
        object_info = MINIO_CLIENT.stat_object(
            bucket_name = bucket_name,
            object_name = object_path + '.pkl'
        )      
        return True
    except Exception as e:
        return False 
# Works
def delete_object(
    logger: any,
    minio_client: any,
    bucket_name: str, 
    object_path: str
) -> bool: 
    MINIO_CLIENT = minio_client
    try:
        MINIO_CLIENT.remove_object(
            bucket_name = bucket_name, 
            object_name = object_path + '.pkl'
        )
        return True
    except Exception as e:
        logger.error('MinIO object deletion error')
        logger.error(e)
        return False
# Works
def update_object(
    logger: any,
    minio_client: any,
    bucket_name: str, 
    object_path: str, 
    data: any,
    metadata: dict
) -> bool:  
    remove = delete_object(logger,minio_client,bucket_name, object_path)
    if remove:
        create = create_object(logger,minio_client, bucket_name, object_path, data, metadata)
        if create:
            return True
    return False
# works
def create_or_update_object(
    logger: any,
    minio_client: any,
    bucket_name: str, 
    object_path: str, 
    data: any, 
    metadata: dict
) -> any:
    bucket_status = check_bucket(logger,minio_client,bucket_name)
    if not bucket_status:
        creation_status = create_bucket(logger,minio_client,bucket_name)
        if not creation_status:
            return None
    object_status = check_object(logger,minio_client,bucket_name, object_path)
    if not object_status:
        return create_object(logger,minio_client,bucket_name, object_path, data, metadata)
    else:
        return update_object(logger,minio_client,bucket_name, object_path, data, metadata)
# Works
def get_object_data_and_metadata(
    logger: any,
    minio_client: any,
    bucket_name: str, 
    object_path: str
) -> dict:
    MINIO_CLIENT = minio_client
    
    try:
        given_object_info = MINIO_CLIENT.stat_object(
            bucket_name = bucket_name, 
            object_name = object_path + '.pkl'
        )
        # There seems to be some kind of a limit
        # with the amount of request a client 
        # can make, which is why this variable
        # is set here to give more time got the client
        # to complete the request
        given_metadata = given_object_info.metadata
        
        given_object_data = MINIO_CLIENT.get_object(
            bucket_name = bucket_name, 
            object_name = object_path + '.pkl'
        )
        given_pickled_data = given_object_data.data
        
        try:
            given_data = pickle.loads(given_pickled_data)
            relevant_metadata = {} 
            for key, value in given_metadata.items():
                if 'x-amz-meta' in key:
                    key_name = key[11:]
                    relevant_metadata[key_name] = value
            return {'data': given_data, 'metadata': relevant_metadata}
        except Exception as e:
            logger.error('MinIO object pickle decoding error')
            logger.error(e)
            return None 
    except Exception as e:
        logger.error('MinIO object fetching error')
        logger.error(e)
        return None
# Works
def get_object_list(
    logger: any,
    minio_client: any,
    bucket_name: str,
    path_prefix: str
) -> dict:
    MINIO_CLIENT = minio_client
    try:
        objects = MINIO_CLIENT.list_objects(bucket_name = bucket_name, prefix = path_prefix, recursive = True)
        object_dict = {}
        for obj in objects:
            object_name = obj.object_name
            object_info = MINIO_CLIENT.stat_object(
                bucket_name = bucket_name,
                object_name = object_name
            )
            given_metadata = {} 
            for key, value in object_info.metadata.items():
                if 'X-Amz-Meta' in key:
                    key_name = key[11:]
                    given_metadata[key_name] = value
            object_dict[obj.object_name] = given_metadata
        return object_dict
    except Exception as e:
        return None  
# Works
def delete_objects(
    logger: any,
    minio_client: any, 
    bucket_name: str,
    path_prefix: str
):
    objects = get_object_list(logger,minio_client,bucket_name, path_prefix)
    for object_name in objects.keys():
        pkl_split = object_name.split('.')[0]
        delete_object(logger,minio_client,bucket_name,pkl_split)

In [51]:
import io
import pickle
from minio import Minio

def is_minio_client(
    storage_client: any
) -> bool:
    return isinstance(storage_client, Minio)

def setup_minio(
    endpoint: str,
    username: str,
    password: str
) -> any:
    minio_client = Minio(
        endpoint = endpoint, 
        access_key = username, 
        secret_key = password,
        secure = False
    )
    return minio_client

def pickle_data(
    data: any
) -> any:
    pickled_data = pickle.dumps(data)
    length = len(pickled_data)
    buffer = io.BytesIO()
    buffer.write(pickled_data)
    buffer.seek(0)
    return buffer, length

def unpickle_data(
    pickled: any
) -> any:
    return pickle.loads(pickled)

def minio_create_bucket(
    minio_client: any,
    bucket_name: str
) -> bool: 
    try:
        minio_client.make_bucket(
            bucket_name = bucket_name
        )
        return True
    except Exception as e:
        print('MinIO bucket creation error')
        print(e)
        return False
    
def minio_check_bucket(
    minio_client: any,
    bucket_name:str
) -> bool:
    try:
        status = minio_client.bucket_exists(
            bucket_name = bucket_name
        )
        return status
    except Exception as e:
        print('MinIO bucket checking error')
        print(e)
        return False 
       
def minio_delete_bucket(
    minio_client: any,
    bucket_name:str
) -> bool:
    try:
        minio_client.remove_bucket(
            bucket_name = bucket_name
        )
        return True
    except Exception as e:
        print('MinIO bucket deletion error')
        print(e)
        return False
# Works
def minio_create_object(
    minio_client: any,
    bucket_name: str, 
    object_path: str, 
    data: any,
    metadata: dict
) -> bool: 
    # Be aware that MinIO objects have a size limit of 1GB, 
    # which might result to large header error    
    #length = len(data)

    try:
        buffer, length = pickle_data(
            data = data
        )

        minio_client.put_object(
            bucket_name = bucket_name,
            object_name = object_path,
            data = buffer,
            length = length,
            metadata = metadata
        )
        return True
    except Exception as e:
        print('MinIO object creation error')
        print(e)
        return False
# Works
def minio_check_object(
    minio_client: any,
    bucket_name: str, 
    object_path: str
) -> any: 
    try:
        object_info = minio_client.stat_object(
            bucket_name = bucket_name,
            object_name = object_path
        )      
        return object_info
    except Exception as e:
        return {}
# Works
def minio_delete_object(
    minio_client: any,
    bucket_name: str, 
    object_path: str
) -> bool: 
    try:
        minio_client.remove_object(
            bucket_name = bucket_name, 
            object_name = object_path
        )
        return True
    except Exception as e:
        print('MinIO object deletion error')
        print(e)
        return False
# Works
def minio_update_object(
    minio_client: any,
    bucket_name: str, 
    object_path: str, 
    data: any,
    metadata: dict
) -> bool:  
    remove = minio_delete_object(
        minio_client = minio_client,
        bucket_name = bucket_name,
        object_path = object_path
    )
    if remove:
        return minio_create_object(
            minio_client = minio_client, 
            bucket_name = bucket_name, 
            object_path = object_path, 
            data = data,
            metadata = metadata
        )
    return False
# works
def minio_create_or_update_object(
    minio_client: any,
    bucket_name: str, 
    object_path: str, 
    data: any, 
    metadata: dict
) -> bool:
    bucket_status = minio_check_bucket(
        minio_client = minio_client,
        bucket_name = bucket_name
    )
    if not bucket_status:
        creation_status = minio_create_bucket(
            minio_client = minio_client,
            bucket_name = bucket_name
        )
        if not creation_status:
            return False
    object_status = minio_check_object(
        minio_client = minio_client,
        bucket_name = bucket_name, 
        object_path = object_path
    )
    if not object_status:
        return minio_create_object(
            minio_client = minio_client,
            bucket_name = bucket_name, 
            object_path = object_path, 
            data = data, 
            metadata = metadata
        )
    else:
        return minio_update_object(
            minio_client = minio_client,
            bucket_name = bucket_name, 
            object_path = object_path, 
            data = data, 
            metadata = metadata
        )
# Works
def minio_get_object_list(
    minio_client: any,
    bucket_name: str,
    path_prefix: str
) -> any:
    try:
        objects = minio_client.list_objects(
            bucket_name = bucket_name, 
            prefix = path_prefix, 
            recursive = True
        )
        return objects
    except Exception as e:
        return None  
    
def minio_get_object_data_and_metadata(
    minio_client: any,
    bucket_name: str, 
    object_path: str
) -> any:
    try:
        given_object_data = minio_client.get_object(
            bucket_name = bucket_name, 
            object_name = object_path
        )
        
        # There seems to be some kind of a limit
        # with the amount of request a client 
        # can make, which is why this variable
        # is set here to give more time got the client
        # to complete the request

        given_data = unpickle_data(
            pickled = given_object_data.data
        )
        
        #given_data = given_object_data.data

        given_object_info = minio_client.stat_object(
            bucket_name = bucket_name, 
            object_name = object_path
        )
        
        given_metadata = given_object_info.metadata
        
        return {'data': given_data, 'metadata': given_metadata}
    except Exception as e:
        print('MinIO object fetching error')
        print(e)
        return None

## Storing Vectors

## Langchain Functions

In [16]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_huggingface import HuggingFaceEmbeddings

def langchain_create_code_chunks(
    language: any,
    chunk_size: int,
    chunk_overlap: int,
    document: any
) -> any:
    splitter = RecursiveCharacterTextSplitter.from_language(
        language = language,
        chunk_size = chunk_size, 
        chunk_overlap = chunk_overlap
    )

    code_chunks = splitter.create_documents([document])
    code_chunks = [doc.page_content for doc in code_chunks]
    return code_chunks

def lanchain_create_text_chunks(
    chunk_size: int,
    chunk_overlap: int,
    document: any
) -> any:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, 
        chunk_overlap = chunk_overlap,
        length_function = len,
        is_separator_regex = False
    )

    text_chunks = splitter.create_documents([document])
    text_chunks = [doc.page_content for doc in text_chunks]
    return text_chunks

def langchain_create_chunk_embeddings(
    model_name: str,
    chunks: any
) -> any:
    embedding_model = HuggingFaceEmbeddings(
        model_name = model_name
    )
    chunk_embeddings = embedding_model.embed_documents(
        texts = chunks
    )
    return chunk_embeddings

## Qdrant Functions

In [17]:
from qdrant_client import QdrantClient as qc
from qdrant_client import models

def qdrant_is_client(
    storage_client: any
) -> any:
    try:
        return isinstance(storage_client, qc.Connection)
    except Exception as e:
        return False

def qdrant_setup_client(
    api_key: str,
    address: str, 
    port: str
) -> any:
    try:
        qdrant_client = qc(
            host = address,
            port = int(port),
            api_key = api_key,
            https = False
        ) 
        return qdrant_client
    except Exception as e:
        return None

def qdrant_create_collection(
    qdrant_client: any, 
    collection_name: str,
    configuration: any
) -> any:
    try:
        result = qdrant_client.create_collection(
            collection_name = collection_name,
            vectors_config = configuration
        )
        return result
    except Exception as e:
        print(e)
        return None

def qdrant_get_collection(
    qdrant_client: any, 
    collection_name: str
) -> any:
    try:
        collection = qdrant_client.get_collection(
            collection_name = collection_name
        )
        return collection
    except Exception as e:
        return None

def qdrant_list_collections(
    qdrant_client: any
) -> any:
    try:
        collections = qdrant_client.get_collections()
        collection_list = []
        for description in collections.collections:
            collection_list.append(description.name)
        return collection_list
    except Exception as e:
        return []

def qdrant_remove_collection(
    qdrant_client: any, 
    collection_name: str
) -> bool:
    try:
        qdrant_client.delete_collection(collection_name)
        return True
    except Exception as e:
        return False

def qdrant_upsert_points(
    qdrant_client: qc, 
    collection_name: str,
    points: any
) -> any:
    try:
        results = qdrant_client.upsert(
            collection_name = collection_name, 
            points = points
        )
        return results
    except Exception as e:
        print(e)
        return None

def qdrant_search_data(
    qdrant_client: qc,  
    collection_name: str,
    scroll_filter: any,
    limit: str
) -> any:
    try:
        hits = qdrant_client.scroll(
            collection_name = collection_name,
            scroll_filter = scroll_filter,
            limit = limit
        )
        return hits
    except Exception as e:
        print(e)
        return []

def qdrant_search_vectors(
    qdrant_client: qc,  
    collection_name: str,
    query_vector: any,
    limit: str
) -> any:
    try:
        hits = qdrant_client.search(
            collection_name = collection_name,
            query_vector = query_vector,
            limit = limit
        )
        return hits
    except Exception as e:
        return []

def qdrant_remove_vectors(
    qdrant_client: qc,  
    collection_name: str, 
    vectors: str
) -> bool:
    try:
        results = qdrant_client.delete_vectors(
            collection_name = collection_name,
            vectors = vectors
        )
        return results
    except Exception as e:
        print(f"Error removing document: {e}")
        return None

## Vector Functions

In [44]:
import hashlib
import numpy as np
import uuid
import re
from qdrant_client.models import VectorParams, Distance
from qdrant_client.models import PointStruct

def create_document_packet(
    document: any,
    configuration: any,
) -> any:
    document_type = document['type']
    used_configuration = configuration[document_type]
    
    document_chunks = []
    if document_type == 'python':
        document_chunks = langchain_create_code_chunks(
            language = used_configuration['language'],
            chunk_size = used_configuration['chunk-size'],
            chunk_overlap = used_configuration['chunk-overlap'],
            document = document['data']
        )
    if document_type == 'text' or document_type == 'yaml' or document_type == 'markdown':
        document_chunks = lanchain_create_text_chunks(
            chunk_size = used_configuration['chunk-size'],
            chunk_overlap = used_configuration['chunk-overlap'],
            document = document['data']
        )
    # This needs to remove empty chunks
    filtered_chunks = []
    for chunk in document_chunks:
        if chunk.strip():
            filtered_chunks.append(chunk)
        
    vector_embedding = langchain_create_chunk_embeddings(
        model_name = used_configuration['model-name'],
        chunks = filtered_chunks
    )

    packet = {
        'chunks': filtered_chunks,
        'embeddings': vector_embedding
    }
    
    return packet

def format_chunk(
    document_chunk: any
) -> any:
    chunk = re.sub(r'[^\w\s]', '', document_chunk)
    chunk = re.sub(r'\s+', ' ', chunk) 
    chunk = chunk.strip()
    chunk = chunk.lower()
    # This helps to remove unique hashes for duplicates such as:
    # task_id = task_id )
    # task_id = task_id 
    # task_id = task_id )
    return chunk

def generate_chunk_hash(
    document_chunk: any
) -> any:
    cleaned_chunk = format_chunk(
        document_chunk = document_chunk
    )
    return hashlib.md5(cleaned_chunk.encode('utf-8')).hexdigest()

def generate_document_vectors(
    qdrant_client: any,
    document_database: any,
    document_collection: any,
    document_type: any,
    document_id: str, 
    document_chunks: any,
    document_embeddings: any,
    vector_collection: any
):
    vector_points = []
    vector_index = 0
    added_hashes = []
    for chunk in document_chunks:
        vector_id = document_id + '-' + str(vector_index + 1)
        vector_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, vector_id))

        chunk_hash = generate_chunk_hash(
            document_chunk = chunk
        )
        
        existing_chunks = qdrant_search_data(
            qdrant_client = qdrant_client,
            collection_name = vector_collection,
            scroll_filter = models.Filter(
                must = [
                    models.FieldCondition(
                        key = 'chunk_hash',
                        match = models.MatchValue(
                            value = chunk_hash
                        )
                    )
                ]
            ),
            limit = 1
        )
        # Removes duplicates
        if len(existing_chunks[0]) == 0:
            if not chunk_hash in added_hashes:
                given_vector = document_embeddings[vector_index]

                chunk_point = PointStruct(
                    id = vector_uuid, 
                    vector = given_vector,
                    payload = {
                        'database': document_database,
                        'collection': document_collection,
                        'document': document_id,
                        'type': document_type,
                        'chunk': chunk,
                        'chunk_hash': chunk_hash
                    }
                )
                added_hashes.append(chunk_hash)
                vector_points.append(chunk_point)
        vector_index += 1
    return vector_points

def create_document_vectors(
    qdrant_client: any,
    document_database,
    document_collection,
    document: any,
    configuration: any,
    vector_collection: str
) -> bool:
    document_id = str(document['_id'])
    document_type = document['type']

    document_packet = {}
    try:
        document_packet = create_document_packet(
            document = document,
            configuration = configuration
        )
    except Exception as e:
        print(document_database,document_collection,document_id)
        print(e)

    if 0 == len(document_packet):
        return []
        
    document_chunks = document_packet['chunks']
    document_embeddings = document_packet['embeddings']
    
    if 0 == len(document_embeddings):
        return []
    
    vector_collections = qdrant_list_collections(
        qdrant_client = qdrant_client
    )
    
    collection_created = None
    if not vector_collection in vector_collections:
        collection_configuration = VectorParams(
            size = len(document_embeddings[0]), 
            distance = Distance.COSINE
        )
        collection_created = qdrant_create_collection(
            qdrant_client = qdrant_client,
            collection_name = vector_collection,
            configuration = collection_configuration
        )

    vector_points = generate_document_vectors(
        qdrant_client = qdrant_client,
        document_database = document_database,
        document_collection = document_collection,
        document_type = document_type,
        document_id = document_id,
        document_chunks = document_chunks,
        document_embeddings = document_embeddings,
        vector_collection = vector_collection
    )

    return vectors

def store_vectors(
    qdrant_client: any,
    configuration: any,
    storage_documents: any
):
    print('Storing vectors')
    # create progress logs

    used_object_bucket = 'llm-rag'
    used_object_path = 'vector-documents'
    
    identities_exists = minio_check_object(
        minio_client = minio_client,
        bucket_name = used_object_bucket, 
        object_path = used_object_path
    )

    document_identities = []
    if not len(identities_exists) == 0:
        document_identities = minio_get_object_data_and_metadata(
            minio_client = minio_client,
            bucket_name = used_object_bucket, 
            object_path = used_object_path
        )['data']
    
    amount_of_databases = len(storage_documents)
    database_index = 1
    for document_database, document_collections in storage_documents.items():
        vector_collection = document_database.replace('|','-') + '-embeddings'
        amount_of_collections = len(document_collections)
        collection_index = 1
        database_vectors = []
        for document_collection, documents in document_collections.items():
            amount_of_documents = len(documents)
            for document in documents:
                document_identity = document_database + '-' + document_collection + '-' + str(document['_id'])

                if document_identity in document_identities:
                    continue
                    
                document_vectors = create_document_vectors(
                    qdrant_client = qdrant_client,
                    document_database = document_database,
                    document_collection = document_collection,
                    document = document,
                    configuration = configuration,
                    vector_collection = vector_collection
                )
                database_vectors.extend(document_vectors)
                document_identities.append(document_identity)
            print('Collections: ' + str(collection_index) + '|' + str(amount_of_collections))
            collection_index += 1
        points_stored = qdrant_upsert_points(
            qdrant_client = qdrant_client, 
            collection_name = vector_collection,
            points = database_vectors
        )
        print('Databases: ' + str(database_index) + '|' + str(amount_of_databases))
        database_index += 1

    minio_create_or_update_object(
        minio_client = minio_client,
        bucket_name = used_object_bucket, 
        object_path = used_object_path
        data = document_identities, 
        metadata = {}
    )
    
    print('Vectors stored')

In [21]:
minio_client = setup_minio(
    endpoint = '127.0.0.1:9000',
    username = 'minio123',
    password = 'minio456'
)

In [19]:
qdrant_client = qdrant_setup_client(
    api_key = 'qdrant_key',
    address = '127.0.0.1', 
    port = '6333'
)

/home/sfniila/Project/cloud-hpc-oss-mlops-platform/applications/development/LLMs/pipeline/llm_venv/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [20]:
vector_configuration = {
    'python': {
        'language': Language.PYTHON,
        'chunk-size': 50,
        'chunk-overlap': 0,
        'model-name': 'sentence-transformers/all-MiniLM-L6-v2'
    },
    'markdown': {
        'chunk-size': 50,
        'chunk-overlap': 0,
        'model-name': 'sentence-transformers/all-MiniLM-L6-v2'
    },
    'yaml': {
        'chunk-size': 50,
        'chunk-overlap': 0,
        'model-name': 'sentence-transformers/all-MiniLM-L6-v2'
    }
}

In [ ]:
store_vectors(
    qdrant_client = qdrant_client,
    configuration = vector_configuration,
    storage_documents = stored_documents
)

## Keywords

## SpaCy functions

In [22]:
import spacy

nlp = spacy.load("en_core_web_sm")

def get_document_keywords(
    document: str
):
    doc = nlp(document.lower())
    
    keywords = [
        token.lemma_ for token in doc
        if not token.is_stop               
        and not token.is_punct              
        and not token.is_space              
        and len(token) > 1                  
    ]
    
    keywords = list(set(keywords))
    
    return keywords

## Meili Functions

In [23]:
import meilisearch as ms

def meili_is_client(
    storage_client: any
) -> any:
    try:
        return isinstance(storage_client, ms.Connection)
    except Exception as e:
        print(e)
        return False

def meili_setup_client(
    host: str, 
    api_key: str
) -> any:
    try:
        meili_client = ms.Client(
            url = host, 
            api_key = api_key
        )
        return meili_client 
    except Exception as e:
        print(e)
        return None

def meili_get_index( 
    meili_client: any, 
    index_name: str
) -> any:
    try:
        index = meili_client.index(
            uid = index_name
        )
        return index
    except Exception as e:
        print(e)
        return None
    
def meili_check_index(
    meili_client: any, 
    index_name: str
) -> bool:
    try:
        meili_client.get_index(
            uid = index_name
        )
        return True
    except Exception as e:
        print(e)
        return False
    
def meili_remove_index(
    meili_client: any, 
    index_name: str
) -> bool:
    try:
        response = meili_client.index(
            index_name = index_name
        ).delete()
        return response
    except Exception as e:
        print(e)
        return None
    
def meili_list_indexes(
    meili_client: any
) -> bool:
    try:
        indexes = meili_client.get_indexes()
        return indexes
    except Exception as e:
        print(e)
        return None

def meili_add_documents(
    meili_client: any, 
    index_name: str, 
    documents: any
) -> any:
    try:
        index = meili_get_index(
            meili_client = meili_client,
            index_name = index_name
        )
        response = index.add_documents(
            documents = documents
        )
        return response
    except Exception as e:
        print(e)
        return None

def meili_set_filterable(
    meili_client: any, 
    index_name: str, 
    attributes: any
) -> any:
    try:
        index = meili_get_index(
            meili_client = meili_client,
            index_name = index_name
        )
        response = index.update_filterable_attributes(attributes)
        return response
    except Exception as e:
        print(e)
        return None

def meili_search_documents(
    meili_client: any, 
    index_name: str, 
    query: any, 
    options: any
) -> any:
    try:
        index = meili_get_index(
            meili_client = meili_client,
            index_name = index_name
        )
        response = index.search(
            query,
            options
        )
        return response
    except Exception as e:
        print(e)
        return None
    
def meili_update_documents(
    meili_client, 
    index_name, 
    documents
) -> any:
    try:
        index = meili_client.index(
            index_name = index_name
        )
        response = index.update_documents(
            documents = documents
        )
        return response
    except Exception as e:
        print(e)
        return None

def meili_delete_documents(
    meili_client: any, 
    index_name: str, 
    ids: any
) -> any:
    try:
        index = meili_client.index(
            index_name = index_name
        )
        response = index.delete_documents(
            document_ids = ids
        )
        return response
    except Exception as e:
        print(e)
        return None

## Keyword Functions

In [68]:
import uuid

def generate_keyword_uuid(
    document_id: str,
    document_index: int
) -> str:
    keyword_id = document_id + '-' + str(document_index + 1)
    keyword_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, keyword_id))
    return keyword_uuid

def create_document_keywords(
    document_database: str,
    document_collection: str,
    document: any,
    document_index: int
) -> any:
    document_id = str(document['_id'])
    document_data = document['data']
    document_type = document['type']
    
    document_keywords = get_document_keywords(
        document = document_data
    )
    
    keyword_uuid = generate_keyword_uuid(
        document_id = document_id,
        document_index = document_index
    ) 

    payload = {
        'id': keyword_uuid,
        'database': document_database,
        'collection': document_collection,
        'document': document_id,
        'type': document_type,
        'keywords': document_keywords
    }

    return payload
    
def store_keywords(
    minio_client: any,
    meili_client: any,
    storage_documents: any
):

    used_object_bucket = 'llm-rag'
    used_object_path = 'search-documents'
    
    identities_exists = minio_check_object(
        minio_client = minio_client,
        bucket_name = used_object_bucket, 
        object_path = used_object_path
    )

    document_identities = []
    if not len(identities_exists) == 0:
        document_identities = minio_get_object_data_and_metadata(
            minio_client = minio_client,
            bucket_name = used_object_bucket, 
            object_path = used_object_path
        )['data']

    amount_of_databases = len(storage_documents)
    database_index = 1
    for document_database, collections in storage_documents.items():
        keyword_collection = document_database.replace('|','-') + '-keywords'
        database_keywords = []
        collection_index = 1
        amount_of_collections = len(collections)
        for document_collection, documents in collections.items():
            document_index = 1
            for document in documents:
                document_identity = document_database + '-' + document_collection + '-' + str(document['_id'])

                if document_identity in document_identities:
                    continue

                document_keywords = create_document_keywords(
                    document_database = document_database,
                    document_collection = document_collection,
                    document = document,
                    document_index = document_index
                )

                database_keywords.append(document_keywords)
                
                document_identities.append(document_identity)
            print('Collections: ' + str(collection_index) + '|' + str(amount_of_collections))
            collection_index += 1
        #print(database_keywords)
        stored = meili_add_documents(
            meili_client = meili_client,
            index_name = keyword_collection,
            documents = database_keywords
        )
        print('Databases: ' + str(database_index) + '|' + str(amount_of_databases))
        database_index += 1
        
    minio_create_or_update_object(
        minio_client = minio_client,
        bucket_name = used_object_bucket, 
        object_path = used_object_path,
        data = document_identities, 
        metadata = {}
    )

In [67]:
meili_client.index('K123AsJ0k1-cloud-hpc-oss-mlops-platform-ipynb-keywords').delete()

TaskInfo(task_uid=36, index_uid='K123AsJ0k1-cloud-hpc-oss-mlops-platform-ipynb-keywords', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2024, 11, 1, 13, 51, 22, 24182))

In [25]:
meili_client = meili_setup_client(
    host = 'http://127.0.0.1:7700', 
    api_key = 'meili_key'
)

In [69]:
store_keywords(
    minio_client = minio_client,
    meili_client = meili_client,
    storage_documents = stored_documents
)

Collections: 1|16
Collections: 2|16
Collections: 3|16
Collections: 4|16
Collections: 5|16
Collections: 6|16
Collections: 7|16
Collections: 8|16
Collections: 9|16
Collections: 10|16
Collections: 11|16
Collections: 12|16
Collections: 13|16
Collections: 14|16
Collections: 15|16
Collections: 16|16
Databases: 1|4
Collections: 1|84
Collections: 2|84
Collections: 3|84
Collections: 4|84
Collections: 5|84
Collections: 6|84
Collections: 7|84
Collections: 8|84
Collections: 9|84
Collections: 10|84
Collections: 11|84
Collections: 12|84
Collections: 13|84
Collections: 14|84
Collections: 15|84
Collections: 16|84
Collections: 17|84
Collections: 18|84
Collections: 19|84
Collections: 20|84
Collections: 21|84
Collections: 22|84
Collections: 23|84
Collections: 24|84
Collections: 25|84
Collections: 26|84
Collections: 27|84
Collections: 28|84
Collections: 29|84
Collections: 30|84
Collections: 31|84
Collections: 32|84
Collections: 33|84
Collections: 34|84
Collections: 35|84
Collections: 36|84
Collections: 37

ValueError: [E088] Text of length 1409376 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [47]:
meili_client.index('llm-rag-code-keywords').delete()

TaskInfo(task_uid=32, index_uid='llm-rag-code-keywords', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2024, 11, 1, 13, 29, 58, 264698))

## Hybrid Search Functions

In [ ]:
def clean_prompt(
    prompt: str
) -> any:
    prompt = prompt.lower()
    prompt = re.sub(r'\s+', ' ', prompt)
    prompt = re.sub(r'[^\w\s]', '', prompt)
    return prompt.strip()

def generate_prompt_embedding_query(
    model_name: str,
    prompt: any
) -> any:
    embedding_model = HuggingFaceEmbeddings(
        model_name = model_name
    )
    embedding = embedding_model.embed_documents(
        texts = [prompt]
    )
    return embedding[0]

def spacy_find_keywords(
    text: str
):
    formatted = nlp(text.lower())
    
    keywords = [
        token.lemma_ for token in formatted
        if not token.is_stop               
        and not token.is_punct              
        and not token.is_space              
        and len(token) > 1                  
    ]
    
    keywords = list(set(keywords))
    
    return keywords

def generate_prompt_keyword_query(
    prompt: any
) -> any:
    keywords = spacy_find_keywords(
        text = prompt
    )
    keyword_query = ' OR '.join([f'keywords = "{keyword}"' for keyword in keywords])
    return keyword_query

def calculate_keyword_score(
    keyword_query: str,
    keyword_list: any
) -> any:
    match = 0
    asked_keywords = keyword_query.split('OR')
    for asked_keyword in asked_keywords:
        formatted = asked_keyword.replace('keywords =', '')
        formatted = formatted.replace('"', '')
        formatted = formatted.replace(' ', '')
        
        if formatted in keyword_list:
            match += 1
            
    query_length = len(asked_keywords)
    keyword_length = len(keyword_list)

    if match == 0:
        return 0.0

    normalized = match / ((query_length * keyword_length) ** 0.5)
    return normalized
    
def vector_search_collection(
    vector_client: any,
    search_client: any,
    prompt: str,
    top_k: int
):

    cleaned_prompt = clean_prompt(
        prompt = prompt
    )

    prompt_embedding_query = generate_prompt_embedding_query(
        model_name = 'sentence-transformers/all-MiniLM-L6-v2',
        prompt = cleaned_prompt
    )

    prompt_keyword_query = generate_prompt_keyword_query(
        prompt = cleaned_prompt
    )

    qdrant_collections = [
        'llm-rag-code-embeddings',
        'llm-rag-workflows-embeddings'
    ]
    
    recommeded_cases = []
    for collection in qdrant_collections:
        results = qdrant_search_vectors(
            qdrant_client = vector_client,  
            collection_name = collection,
            query_vector = prompt_embedding_query,
            limit = top_k
        ) 
        
        for result in results:
            res_database = result.payload['database']
            res_collection = result.payload['collection']
            res_document = result.payload['document']
            res_type = result.payload['type']
            res_score = result.score
            
            res_case = {
                'source': 'vector',
                'database': res_database,
                'collection': res_collection,
                'document': res_document,
                'type': res_type,
                'score': res_score
            }
            
            recommeded_cases.append(res_case)
            
    meili_collections = [
        'llm-rag-code-keywords',
        'llm-rag-workflows-keywords'
    ]

    recommeded_keyword_cases = []
    for index in meili_collections:
        results = meili_search_documents(
            meili_client = search_client, 
            index_name = index, 
            query = "", 
            options = {
                'filter': prompt_keyword_query,
                'attributesToRetrieve': ['database','collection','document', 'keywords'],
                'limit': top_k
            }
        )

        for result in results['hits']:
            res_database = result['database']
            res_collection = result['collection']
            res_document = result['document']
            res_keywords = result['keywords']
            
            res_score = calculate_keyword_score(
                keyword_query = prompt_keyword_query,
                keyword_list = res_keywords
            )

            res_case = {
                'source': 'search',
                'database': res_database,
                'collection': res_collection,
                'document': res_document,
                'type': res_type,
                'score': res_score
            }

            recommeded_cases.append(res_case)

    return recommeded_cases

def get_top_document_metadata(
    collection: str,
    alpha: float
) -> any:
    df = pd.DataFrame(collection)
    ids_with_both = df.groupby('document')['source'].nunique()
    ids_with_both = ids_with_both[ids_with_both > 1].index
    filtered_df = df[df['document'].isin(ids_with_both)]

    matched_documents = []
    for index_i, row_i in filtered_df[filtered_df['source'] == 'vector'].iterrows():
        vector_source = row_i['source']
        vector_database = row_i['database']
        vector_collection = row_i['collection']
        vector_id = row_i['document']
        vector_type = row_i['type']
        vector_score = row_i['score']
        
        for index_j, row_j in filtered_df[filtered_df['source'] == 'search'].iterrows():
            search_source = row_j['source']
            search_database = row_j['database']
            search_collection = row_j['collection']
            search_id = row_j['document']
            search_type = row_j['type']
            search_score = row_j['score']
            
            if vector_database == search_database:
                if vector_collection == search_collection:
                    if vector_type == search_type:
                        if vector_id == search_id:
                            hybrid_score = vector_score * alpha + search_score * (1-alpha)
    
                            matched_documents.append({
                                'source': 'hybrid',
                                'database': search_database,
                                'collection': search_collection,
                                'document': search_id,
                                'score': hybrid_score
                            })
    
    match_df = pd.DataFrame(matched_documents)
    print(match_df)
    return match_df.nlargest(1, 'score').values.tolist()[0]